In [1]:
from importlib import reload
import numpy as np
import numpy.typing as npt
import os
import pandas as pd
import pickle
from pprint import pprint
import tensorflow as tf
tf.keras.backend.set_floatx('float64')
from tensorflow import keras
from tqdm import tqdm
from typing import Callable, Generator, List, Tuple

from src.harness import architecture as arch
from src.harness import dataset as ds
from src.harness import history as hist
from src.harness import utils

2025-02-22 13:43:17.561730: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-22 13:43:17.580700: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-22 13:43:17.600523: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-22 13:43:17.606750: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-22 13:43:17.624041: I tensorflow/core/platform/cpu_feature_guar

In [2]:
path = os.path.join(os.path.expanduser("~"), "lottery-tickets/experiments/11-04-2024/")
experiment_directories = os.listdir(path)
pprint(experiment_directories)
e = os.path.join(path, experiment_directories[1])
print(e)
# experiments = list(hist.get_experiments(e))
# e0 = experiments[0]
# for t1 in experiments[0]:
#     pass

# a = arch.Architecture("lenet", "mnist")
# model = a.get_model_constructor()()
# model.set_weights([m * w for m, w in zip(t1.masks, t1.final_weights)])
# shape = [1] + list(model.input_shape[1:])
# model.trainable_variables


['lenet_fashion_mnist_0_seed_5_experiments_1_batches_0.025_default_sparsity_lm_pruning_20241102-111614',
 'lenet_mnist_0_seed_5_experiments_1_batches_0.025_default_sparsity_lm_pruning_20241102-111614',
 'lenet_cifar_0_seed_5_experiments_1_batches_0.025_default_sparsity_lm_pruning_20241102-111614',
 'conv2_cifar_0_seed_5_experiments_1_batches_0.025_default_sparsity_lm_pruning_20241102-111614',
 'conv6_fashion_mnist_0_seed_5_experiments_1_batches_0.025_default_sparsity_lm_pruning_20241102-111614',
 'conv6_mnist_0_seed_5_experiments_1_batches_0.025_default_sparsity_lm_pruning_20241102-111614',
 'conv4_cifar_0_seed_5_experiments_1_batches_0.025_default_sparsity_lm_pruning_20241102-111614',
 'conv4_fashion_mnist_0_seed_5_experiments_1_batches_0.025_default_sparsity_lm_pruning_20241102-111614',
 'conv2_fashion_mnist_0_seed_5_experiments_1_batches_0.025_default_sparsity_lm_pruning_20241102-111614',
 'conv6_cifar_0_seed_5_experiments_1_batches_0.025_default_sparsity_lm_pruning_20241102-111614'

In [43]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Dense
from typing import List, Dict

def compute_synflow_per_weight(net: keras.Model) -> List[tf.Tensor]:
        
    def linearize() -> Dict[str, tf.Tensor]:
        signs = {}
        for layer in net.trainable_variables:
            if hasattr(layer, 'kernel'):
                signs[layer.name] = tf.sign(layer.kernel)
                layer.kernel.assign(tf.abs(layer.kernel))
        return signs

    def nonlinearize(signs: Dict[str, tf.Tensor]) -> None:
        for layer in net.trainable_variables:
            if hasattr(layer, 'kernel'):
                layer.kernel.assign(layer.kernel * signs[layer.name])
    
    signs = linearize()

    input_dim = [1] + list(net.input_shape[1:])
    inputs = tf.ones(input_dim, dtype=tf.float64)
    with tf.GradientTape() as tape:
        output = net(inputs)
        loss = tf.reduce_sum(output)
    grads = tape.gradient(loss, net.trainable_variables)
    scores = [l * g for l, g in zip(net.trainable_variables, grads)]

    nonlinearize(signs)
    
    return scores

def get_layer_metric_array(model: keras.Model, metric: Callable) -> List[tf.Tensor]:
    metric_array = []
    for layer in model.layers:
        if isinstance(layer, Conv2D) or isinstance(layer, Dense) and layer.activation == tf.keras.activations.linear:
            metric_array.append(metric(layer))
        else:
            metric_array.append([])
    return metric_array

a = arch.Architecture("lenet", "mnist")
rando = a.get_model_constructor()()

compute_synflow_per_weight(rando)


[<tf.Tensor: shape=(784, 300), dtype=float64, numpy=
 array([[ 0.00000000e+00, -0.00000000e+00, -0.00000000e+00, ...,
         -0.00000000e+00,  1.69993644e-20,  7.65023895e-20],
        [ 0.00000000e+00, -0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00, -2.40102756e-20, -1.23518654e-19],
        [ 0.00000000e+00, -0.00000000e+00,  0.00000000e+00, ...,
         -0.00000000e+00,  1.55325454e-21,  9.59051192e-20],
        ...,
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  2.56917133e-20,  1.71611590e-19],
        [-0.00000000e+00,  0.00000000e+00, -0.00000000e+00, ...,
         -0.00000000e+00,  1.05544929e-22, -1.15812276e-19],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  3.38390952e-21, -7.02143013e-20]])>,
 <tf.Tensor: shape=(300,), dtype=float64, numpy=
 array([-0.,  0., -0.,  0., -0.,  0., -0., -0.,  0., -0.,  0.,  0.,  0.,
         0., -0., -0., -0.,  0., -0., -0.,  0.,  0., 

In [17]:
def make_layer_features(
    architecture: str, 
    initial: List[npt.NDArray], 
    final: List[npt.NDArray], 
    masks: List[npt.NDArray],
) -> pd.DataFrame:
    layer_features = {
        "l_num": [],
        "l_size": [],
        "l_rel_size": [],
        "l_sparsity": [],
        "lf_mean": [],
        "lf_std": [],
        "lf_prop_positive": [],
        "li_mean": [],
        "li_std": [],
        "li_prop_positive": [],
    }
     # Loop over layers, vectorize calculations where possible
    print("Creating layer features")
    total_size = sum(map(np.size, masks))
    for index, (iw, fw, m) in tqdm(enumerate(zip(initial, final, masks))):
        print(f"Layer {index}")
        mask = m.astype(bool)
        fw_filtered = fw[mask]
        iw_filtered = iw[mask]
        layer_features["l_num"].append(index) 
        layer_features["l_size"].append(m.size) 
        layer_features["l_rel_size"].append(m.size / total_size) 
        layer_features["l_sparsity"].append(np.mean(m))
        layer_features["lf_mean"].append(np.mean(fw_filtered))
        layer_features["lf_std"].append(np.std(fw_filtered)) 
        layer_features["lf_prop_positive"].append(np.mean(fw_filtered >= 0))  
        layer_features["li_mean"].append(np.mean(iw_filtered))  
        layer_features["li_std"].append(np.std(iw_filtered)) 
        layer_features["li_prop_positive"].append(np.mean(iw_filtered >= 0)) 
    layer_ohe = arch.Architecture.ohe_layer_types(architecture)
    for index, name in enumerate(arch.Architecture.LAYER_TYPES):
        layer_features[name] = [ohe[index] for ohe in layer_ohe]
    layer_df = pd.DataFrame(layer_features)
    
    return layer_df

def make_weight_features(
    layer_df: pd.DataFrame,
    architecture: arch.Architecture,
    initial: List[npt.NDArray],
    final: List[npt.NDArray],
    masks: List[npt.NDArray],
) -> pd.DataFrame:
    # Prepare weight features: No need for large weight_features array, use a list of dicts
    weight_features_list = []
    start_idx = 0

    print("Making weight features")
    model = architecture.get_model_constructor()()
    num_weights = sum(map(np.size, model.get_weights()))
    shape = [1] + list(model.input_shape[1:])
    model.set_weights([w * m for w, m in zip(final, masks)])
    synflow_scores = synflow(model, shape)
    # Duplicate features for initial vs. final weights
    for layer, (iw, fw, m) in tqdm(enumerate(zip(initial, final, masks))):
        print(f"Layer {layer}")
        mask = m.astype(bool).ravel()
        num_params = len(mask)
        num_nonzero = np.count_nonzero(mask)
        num_zero = num_params - num_nonzero
        # Precompute values for this layer
        layer_num = np.full(num_params, layer, dtype=np.int8)
        weight_nums = np.arange(num_params, dtype=np.int32)
        
        f_flat = fw.flatten()
        f_sorted = np.sort(f_flat)
        f_sign = np.sign(f_flat)
        f_mag = np.abs(f_flat, dtype=np.float32)
        f_perc = np.array([np.argmax(v < f_sorted) - num_zero for v in f_flat]) / num_nonzero
        f_norm_std = (f_flat - layer_df["lf_mean"].iloc[layer]) / layer_df["lf_std"].iloc[layer]
        
        
        i_flat = iw.flatten()
        i_sorted = np.sort(i_flat)
        i_sign = np.sign(i_flat)
        i_mag = np.abs(i_flat, dtype=np.float32)
        i_perc = np.array([np.argmax(v < i_sorted) - num_zero for v in i_flat]) / num_nonzero
        i_norm_std = (f_flat - layer_df["li_mean"].iloc[layer]) / layer_df["li_std"].iloc[layer]
         
        # Create a dictionary for weight features for this layer
        layer_weight_features = {
            "l_num": layer_num,
            "w_num": weight_nums,
            "wf_sign": f_sign,
            "wi_sign": i_sign,
            "wf_val": f_flat,
            "wi_val": i_flat,
            "wf_mag": f_mag,
            "wi_mag": i_mag,
            "wf_perc": f_perc.astype(np.float32),
            "wi_perc": i_perc.astype(np.float32),
            "wf_std": f_norm_std.astype(np.float32),
            "wi_std": i_norm_std.astype(np.float32),
            "w_mask": mask,
            "w_synflow": synflow_scores[layer].numpy().flatten()
        }
        
        weight_features_list.append(pd.DataFrame(layer_weight_features))

    weight_df = pd.concat(weight_features_list, axis=0, ignore_index=True)
    
    return weight_df

In [26]:
def make_trial_features(trial: hist.TrialData, t_num: int, e_num: int) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    # Trial DataFrame
    total_size = sum(map(np.size, trial.masks))
    sparsity = sum(map(np.sum, trial.masks)) / total_size
    trial_df = pd.DataFrame({
        "e_num": [e_num],
        "t_num": [t_num],
        "seed": [trial.random_seed],
        "step": [trial.pruning_step],
        "arch": [trial.architecture],
        "dataset": [trial.dataset],
        "sparsity": [sparsity],
        "size": [total_size],
    })
    trial_df["sparsity"] = trial_df["sparsity"].astype("float32")
    
    def add_labels(df: pd.DataFrame):
        df["e_num"] = e_num
        df["e_num"] = df["e_num"].astype("uint8")
        df["t_num"] = t_num
        df["t_num"] = df["t_num"].astype("uint8")
    
    # Layer feature computation
    layer_df = make_layer_features(trial.architecture, trial.initial_weights, trial.final_weights, trial.masks)
    add_labels(layer_df)
    
    architecture = arch.Architecture(trial.architecture, trial.dataset)
    weight_df = make_weight_features(layer_df, architecture, trial.initial_weights, trial.final_weights, trial.masks)
    add_labels(weight_df)
    
    return trial_df, layer_df, weight_df

def build_exp_dfs(exp: Generator[hist.TrialData, None, None], e_num: int) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:      
    trials = []
    layers = []
    weights = []
    print(f"Building dataframes for experiment {e_num}")
    for t_num, t in tqdm(enumerate(exp)): 
        print(f"Trial {t_num}")
        # Dummy line
        t.seed_weights = lambda x: x
        t_df, l_df, w_df = make_trial_features(t, t_num=t_num, e_num=e_num)
        trials.append(t_df)
        layers.append(l_df)
        weights.append(w_df)

    trials_df = pd.concat(trials, axis=0, ignore_index=True)
    layers_df = pd.concat(layers, axis=0, ignore_index=True)
    weights_df = pd.concat(weights, axis=0, ignore_index=True)

    return trials_df, layers_df, weights_df

def build_dataframes(epath: str) -> Tuple[pd.DataFrame, ...]:
    experiments = list(hist.get_experiments(epath))
    trials = []
    layers = []
    weights = []
    print(f"Building dataframes for experiments at {epath}")
    for e_num, e in tqdm(enumerate(experiments)): 
        if len(trials) >= 2:
            break
        print(f"Experiment {e_num}")
        t_df, l_df, w_df = build_exp_dfs(e, e_num)
        trials.append(t_df)
        layers.append(l_df)
        weights.append(w_df)

    trials_df = pd.concat(trials, axis=0, ignore_index=True)
    layers_df = pd.concat(layers, axis=0, ignore_index=True)
    weights_df = pd.concat(weights, axis=0, ignore_index=True)

    return trials_df, layers_df, weights_df
    
tdf, ldf, wdf = build_dataframes(e)

Building dataframes for experiments at /users/j/b/jbourde2/lottery-tickets/experiments/11-04-2024/lenet_mnist_0_seed_5_experiments_1_batches_0.025_default_sparsity_lm_pruning_20241102-111614


0it [00:00, ?it/s]

Experiment 0
Building dataframes for experiment 0



0it [00:00, ?it/s]

Trial 0
Creating layer features


...



1it [00:21, 21.57s/it]/tmp/ipykernel_116246/1621502057.py:85: RuntimeWarning: divide by zero encountered in divide
  i_norm_std = (f_flat - layer_df["li_mean"].iloc[layer]) / layer_df["li_std"].iloc[layer]
/tmp/ipykernel_116246/1621502057.py:85: RuntimeWarning: invalid value encountered in divide
  i_norm_std = (f_flat - layer_df["li_mean"].iloc[layer]) / layer_df["li_std"].iloc[layer]


Layer 1
Layer 2




6it [00:22,  3.67s/it]

18it [06:28, 21.58s/it]


Layer 3
Layer 4
Layer 5


2it [13:07, 393.95s/it]


In [ ]:
trials_df, layers_df, weights_df = tdf, ldf, wdf

In [ ]:
df_path = "weightabase.pkl"

In [ ]:
def merge_dfs(
    trials_df: pd.DataFrame,
    layers_df: pd.DataFrame,
    weights_df: pd.DataFrame,
) -> pd.DataFrame:
    keys = ["e_num", "t_num", "l_num", "w_num"]
    tl_df = trials_df.merge(layers_df, on=keys[:2])
    tlw_df = tl_df.merge(weights_df, on=keys[:3])
    tlw_df = pd.get_dummies(tlw_df, columns=["arch"])
    tlw_df = pd.get_dummies(tlw_df, columns=["dataset"])
    drop_cols = keys + ["step", "seed"]
    tlw_df.drop(columns=drop_cols, inplace=True)
    return tlw_df

merged_df = merge_dfs(trials_df, layers_df, weights_df)


In [ ]:
merged_df.to_pickle(df_path)

In [ ]:
merged_df = pd.read_pickle(df_path)

In [ ]:
def featurize_db(
    tlw_df: pd.DataFrame,
    features: List[str] = [],
) -> Tuple[npt.NDArray, npt.NDArray]: 
    if not features:
        raise ValueError("No features provided.")
    Y, X = tlw_df["w_mask"], tlw_df[features]
    return X.to_numpy().astype(np.float32), Y.to_numpy()

f_features = ["l_sparsity", "l_rel_size", "lf_prop_positive", "wf_std", "wf_perc", "wf_mag", "wf_sign", "w_mask", "dense", "bias", "conv", "output"]
i_features = ["l_sparsity", "l_rel_size", "li_prop_positive", "wi_std", "wi_perc", "wi_mag", "wi_sign", "w_mask", "dense", "bias", "conv", "output"]
fX, fY = featurize_db(merged_df, f_features)
iX, iY = featurize_db(merged_df, i_features)

In [ ]:
fX[0]

In [ ]:
iX[0]

In [ ]:
def create_meta(shape: Tuple[int, ...]) -> keras.Model:
    model = keras.Sequential([
        keras.Input(shape=shape),
        keras.layers.Dense(8, "relu"),
        keras.layers.Dense(1, "sigmoid"),
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
             loss=tf.keras.losses.BinaryCrossentropy(),
             metrics=["accuracy"])
    return model

# print("Creating meta mask model off the final weights")
# f_meta = create_meta(fX[0].shape)
# f_meta.fit(fX, fY, epochs=3, batch_size=256, validation_split=0.2, shuffle=True)

print("Creating meta mask model off the initial weights")
i_meta = create_meta(iX[0].shape)
i_meta.fit(iX, iY, epochs=3, batch_size=256, validation_split=0.2, shuffle=True)


In [ ]:
f_meta.save_weights("f_meta.weights.h5")
i_meta.save_weights("i_meta.weights.h5")

In [ ]:
import copy
from typing import Callable

def make_meta_mask(
    meta: keras.Model,
    make_x: Callable[[str, str, keras.Model, List[npt.NDArray]], npt.NDArray],
    architecture: str,
    dataset: str,
    steps: int,
) -> Tuple[List[npt.NDArray], List[float]]:
    a = arch.Architecture(architecture, dataset)
    _, val_X, _, val_Y = a.load_data()
    model = a.get_model_constructor()()
    original_weights = copy.deepcopy(model.get_weights())
    model.compile(optimizer="Adam", loss=tf.keras.losses.CategoricalCrossentropy(), metrics=["accuracy"])
    masks = [np.ones_like(w) for w in model.get_weights()]
    def update_masks(mask_pred: npt.NDArray) -> List[npt.NDArray]:
        start = 0
        end = 0
        new_masks = []
        nonlocal masks
        for m in masks:
            end += m.size
            new_m = np.reshape(mask_pred[start:end], m.shape)
            new_masks.append(new_m)
            start = end
        return masks
            
    accuracies = []
    for step in range(steps):
        # Get validation accuracy
        _, accuracy = model.evaluate(val_X, val_Y)
        accuracies.append(accuracy)
        print(f"Step {step} accuracy: {accuracy:.2%}")
        # Extract features
        X = make_x(architecture, dataset, model, masks)
        # Predict and replace existing mask
        mask_pred = meta.predict(X, batch_size=2**20)
        masks = update_masks(mask_pred)
        model.set_weights([w * m for w, m in zip(original_weights, masks)])
        
    return masks, accuracies

def make_x(
    architecture: str,
    dataset: str,
    model: keras.Model,
    masks: List[npt.NDArray],
) -> npt.NDArray:
    # ["l_sparsity", "l_rel_size", "lf_prop_positive", "wf_std", "wf_perc", "wf_mag", "wf_sign", "w_mask", "dense", "bias", "conv", "output"]
    # Layer features
    nparams = sum(map(np.size, masks))
    nfeatures = 10
    features = np.zeros((nparams, nfeatures))
    
    n = 0
    def add_layer_features(layer_values: List[float]):
        nonlocal n
        start = 0
        end = 0
        for v, size in zip(layer_values, map(np.size, masks)):
            end += size
            features[start:end, n] = v
            start = end
        n += 1
        
    def add_weight_features(weight_features: List[npt.NDArray]):
        nonlocal n
        start = 0
        end = 0
        for v in weight_features:
            end += v.size
            features[start:end, n] = np.ravel(v)
            start = end
        n += 1
    
    # Make sure ravel does the right thing in reverse
    masked = [np.ravel(w * m) for w, m in zip(model.get_weights(), masks)]
    sparsities = [np.count_nonzero(m) / np.size(m) for m in masks]
    rel_size = [np.size(m) / nparams for m in masks]
    prop_pos = [np.count_nonzero(w >= 0) for w in masked]
    
    layer_ohe = arch.Architecture.ohe_layer_types(architecture)
    for values in [sparsities, rel_size, prop_pos]:
        add_layer_features(values)
    
    # Weight features
    l_std = [np.std(w) for w in masked]
    l_mean = [np.mean(w) for w in masked]
    l_sorted = [np.sort(np.ravel(w)) for w in masked]
    
    w_std = [(w - l_mean) / l_std for w, l_mean, l_std in zip(l_std, l_mean, masked)]
    w_mag = [np.abs(w) for w in masked]
    w_sign = [np.sign(w) for w in masked]
    num_nonzero = sum(map(np.count_nonzero, masks))
    num_zero = nparams - num_nonzero
    w_perc = np.array([
        np.argmax(np.ravel(v) < v_sorted) - num_zero 
        for v, v_sorted in zip(masked, l_sorted)]
    ) / num_nonzero
    flat_masks = [np.ravel(m) for m in masks]
    for values in [w_std, w_perc, w_mag, w_sign, flat_masks]:
        add_weight_features(values)
    
    for values in [layer_ohe[:, i] for i in range(layer_ohe.shape[1])]:
        add_layer_features(values)
        
    return features
    
masks, accuracies = make_meta_mask(i_meta, make_x, "lenet", "mnist", 50)
